# Sales Account Meeting Briefing

**User Story**: US1 — Meeting Preparation Assistant
**Persona**: Account Executive

This notebook demonstrates how to use the Salesforce AI Assistant to prepare
for account meetings. The assistant pulls live CRM data (account details, contacts,
open opportunities, recent activities) and generates a comprehensive briefing
with recommended talking points.

## Prerequisites
- `.env` file configured with Salesforce and Azure AI credentials
- `salesforce-crm` MCP server available
- Python packages installed (`pip install -r requirements.txt`)

In [1]:
# Cell 2: Environment + Auth Setup
import os
import sys

# Add project root to path
sys.path.insert(0, os.path.abspath(".."))

from dotenv import load_dotenv

load_dotenv("../.env")

# Verify required env vars
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing environment variables: {', '.join(missing)}")

print("✅ Environment configured")
print(f"   Project: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:50]}...")
print(f"   SF Instance: {os.environ['SF_INSTANCE_URL']}")

✅ Environment configured
   Project: https://newfoundrydemoproject.services.ai.azure.co...
   SF Instance: https://orgfarm-181aeeadf6-dev-ed.develop.my.salesforce.com


In [ ]:
# Cell 3: Start MCP Server + Configure Connection
import subprocess
import time

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MCPTool
from azure.identity import DefaultAzureCredential

# Start the salesforce-crm MCP server locally with SSE transport
MCP_PORT = 8100  # Use a dedicated port to avoid conflicts
mcp_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={
        **os.environ,
        "MCP_TRANSPORT": "sse",
        "FASTMCP_PORT": str(MCP_PORT),
    },
    cwd=os.path.abspath(".."),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)  # Wait for server to start

if mcp_process.poll() is not None:
    stderr = mcp_process.stderr.read().decode() if mcp_process.stderr else ""
    raise RuntimeError(f"MCP server failed to start: {stderr}")

MCP_SERVER_URL = f"http://127.0.0.1:{MCP_PORT}/sse"
print(f"✅ MCP server started (PID: {mcp_process.pid}) at {MCP_SERVER_URL}")

# Create AI Project client
project = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Create MCP tool pointing to the local SSE server
mcp_tool = MCPTool(
    server_label="salesforce-crm",
    server_url=MCP_SERVER_URL,
)
print(f"✅ MCP tool configured: {mcp_tool.server_label}")

ImportError: cannot import name 'McpToolConnection' from 'azure.ai.projects.models' (/Users/pradeep330/Documents/aiframework-projects/salesforce-foundryagent/.venv/lib/python3.11/site-packages/azure/ai/projects/models/__init__.py)

In [ ]:
# Cell 4: Create OpenAI client + load system prompt
from pathlib import Path as _Path

# Get an OpenAI client scoped to the Azure AI Foundry project
client = project.get_openai_client()

# Load the Sales agent system prompt
system_prompt = _Path("../agents/sales/system_prompt.md").read_text(encoding="utf-8")

MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ System prompt loaded ({len(system_prompt)} chars)")

In [ ]:
# Cell 5: Meeting Briefing Request
ACCOUNT_NAME = "Acme Corp"  # Change to your target account

response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=f"Prepare me for my meeting with {ACCOUNT_NAME}. "
    f"Include account overview, key contacts, open opportunities, "
    f"recent activities, and suggested talking points.",
    tools=[mcp_tool],
)
print(f"✅ Response completed (id: {response.id})")

In [ ]:
# Cell 6: Display Briefing
from IPython.display import Markdown, display

# Extract text output from the response
for item in response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 7: Follow-up Question (continues conversation via previous_response_id)
response2 = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input="Any other open opportunities with them? What's the total pipeline value?",
    tools=[mcp_tool],
    previous_response_id=response.id,
)

# Display follow-up response
for item in response2.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 8: Cleanup — stop the local MCP server
mcp_process.terminate()
mcp_process.wait(timeout=5)
print(f"✅ MCP server stopped (PID: {mcp_process.pid})")
print("\n--- Session Complete ---")